In [ ]:
import sys
import os

project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.append(project_root)


import numpy as np
import plotly.graph_objects as go
import plotly.io as pio
from plotly.subplots import make_subplots
from constants.constants import exports_dir

# Set default renderer for plotly (optional)
pio.renderers.default = "notebook"

# Constants
a = 3  # starting yield from the treasury balance
r = 0.9  # common ratio
e = 650000  # starting balance of the evil address
b = 10.3136  # starting percentage of spend from the evil address
treasuryShare = 0.77
jackpotShare = 0.20
teamShare = 0.03
total_supply = 1_000_000  # total supply of tokens
mystery_box_share = 0.25  # 25% of the total supply goes to mystery box
public_share = 0.10  # 10% of the total supply goes to the public

# Number of points (n ranges from 1 to 33 with step 1)
n_values = np.arange(1, 34, 1)

# Calculating c and d for each n
c_values = a * (r ** n_values - 1) / (r - 1)
d_values = b * (r ** n_values - 1) / (r - 1)

# Calculating evil address spend for each n (rounded down to integer)
evilAddressSpend_values = np.floor(e * d_values / 100)

# Calculating y for each n (balance of the evil address in the journey)
y_values = e - evilAddressSpend_values

# Calculating jackpot gains for each n
jackpot_gains = evilAddressSpend_values * jackpotShare

# Calculating total balance including jackpot gains
total_balance = jackpot_gains + y_values

# Calculating amount of NFTs bought in each round
amountOfNftsBought = e - y_values

# Calculating amount into the treasury for each n using evilAddressSpend_values * share
amount_into_treasury_values = evilAddressSpend_values * treasuryShare

# Calculating team totals for each n using evilAddressSpend_values * share
team_total_values = evilAddressSpend_values * teamShare

# Calculating yield for each n
yield_values = amount_into_treasury_values * a * (r ** n_values - 1) / (r - 1) / 100

# Updating amount into the treasury by subtracting yield
amount_into_treasury_after_yield = amount_into_treasury_values - yield_values

# Calculating "Locked in NFT" tokens for each n
locked_in_nft_values = yield_values

# Calculating mystery box tokens
mystery_box_tokens = total_supply * mystery_box_share

# Calculating public tokens
public_tokens = total_supply * public_share

# Calculating locked tokens and circulating supply for each n
locked_tokens = amount_into_treasury_after_yield + jackpot_gains + team_total_values + mystery_box_tokens + public_tokens + locked_in_nft_values
circulating_supply = total_supply - locked_tokens

# Calculating evil address spend in each round
evilAddressSpendInEachRound_values = e * b * (r ** (n_values - 1)) / 100

# Initial liquidity pool setup
initial_DARK = total_supply * 0.10  # 10% of the total supply of $DARK in the liquidity pool
initial_DAI = 500_000  # 1,000,000 $DAI in the liquidity pool

# Number of trades per day and total days per n_value
trades_per_day = 100
days_per_n_value = 115
total_days = len(n_values) * days_per_n_value

# Arrays to hold the prices and reserves over time
prices = []
reserves_DARK = [initial_DARK]
reserves_DAI = [initial_DAI]
buy_volumes = []
sell_volumes = []
total_volumes = []
cumulative_volume = 0
cumulative_volumes = []
buy_sell_pressure_index = 1  # Buy/sell pressure index (higher means more buy trades)

# Calculate buy probability
buy_probability = buy_sell_pressure_index / (buy_sell_pressure_index + 1)

# Arrays to hold cumulative buys and sells
cumulative_buys = []
cumulative_sells = []
cumulative_buy_volume = 0
cumulative_sell_volume = 0

# Arrays to hold reserves at the end of each day
daily_reserves_DARK = []
daily_reserves_DAI = []

# Randomly simulate trades for each day
np.random.seed(42)  # For reproducibility
for day in range(total_days):
    daily_buy_volume = 0
    daily_sell_volume = 0
    daily_total_volume = 0
    for _ in range(trades_per_day):
        if np.random.rand() < buy_probability:
            trade_type = "buy"
        else:
            trade_type = "sell"
        trade_size = np.random.uniform(1, 50)  # Randomly choose a trade size between 1 and 50 tokens

        if trade_type == "buy":
            # Buying $DARK with $DAI
            amount_in = trade_size
            reserve_in = reserves_DAI[-1]
            reserve_out = reserves_DARK[-1]
        else:
            # Selling $DARK for $DAI
            amount_in = trade_size
            reserve_in = reserves_DARK[-1]
            reserve_out = reserves_DAI[-1]

        # Uniswap V2 formula: x * y = k, where x and y are the reserves of the two tokens
        amount_out = (amount_in * reserve_out) / (reserve_in + amount_in)

        if trade_type == "buy":
            reserves_DAI.append(reserve_in + amount_in)
            reserves_DARK.append(reserve_out - amount_out)
            daily_buy_volume += amount_in
            cumulative_buy_volume += amount_in
        else:
            reserves_DARK.append(reserve_in + amount_in)
            reserves_DAI.append(reserve_out - amount_out)
            daily_sell_volume += amount_in
            cumulative_sell_volume += amount_in

        daily_total_volume += amount_in

    daily_reserves_DARK.append(reserves_DARK[-1])
    daily_reserves_DAI.append(reserves_DAI[-1])
    cumulative_buys.append(cumulative_buy_volume)
    cumulative_sells.append(cumulative_sell_volume)

    # Calculate the new price after the trade
    new_price = reserves_DAI[-1] / reserves_DARK[-1]
    prices.append(new_price)
    buy_volumes.append(daily_buy_volume)
    sell_volumes.append(daily_sell_volume)
    total_volumes.append(daily_total_volume)
    cumulative_volume += daily_total_volume
    cumulative_volumes.append(cumulative_volume)

# Calculate buy-sell ratio
buy_sell_ratios = np.array(buy_volumes) / (np.array(sell_volumes) + 1)  # Adding 1 to avoid division by zero

# Create a subplot figure with 11 rows and 1 column
fig = make_subplots(rows=11, cols=1, subplot_titles=(
    'Interactive Plot of c and d values',
    'Interactive Plot of y values, Amount of NFTs Bought, Jackpot Gains, and Total Balance',
    'Interactive Plot of Amount into Treasury, Jackpot Gains, Team Total, and Yield',
    'Interactive Plot of Token Distribution (Circulating Supply vs Locked Tokens)',
    'Bar Chart of Evil Address Spend in Each Round',
    'Price of $DARK in terms of $DAI',
    'Sales vs Buys on Each Day',
    'Total Trade Volume and Cumulative Volume on Each Day',
    'Buy-Sell Ratio',
    'Reserves of DARK and DAI Over Time',
    'Cumulative Buys and Sells'
))

# Adding traces for c_values and d_values
fig.add_trace(go.Scatter(x=n_values, y=c_values, mode='lines', name='c values', line=dict(color='red')), row=1, col=1)
fig.add_trace(go.Scatter(x=n_values, y=d_values, mode='lines', name='d values', line=dict(color='green')), row=1, col=1)

# Adding trace for y_values
fig.add_trace(go.Scatter(x=n_values, y=y_values, mode='lines', name='y values', line=dict(color='blue')), row=2, col=1)

# Adding trace for amount of NFTs bought
fig.add_trace(go.Scatter(x=n_values, y=amountOfNftsBought, mode='lines', name='Amount of NFTs Bought', line=dict(color='orange')), row=2, col=1)

# Adding trace for jackpot gains
fig.add_trace(go.Scatter(x=n_values, y=jackpot_gains, mode='lines', name='Jackpot Gains', line=dict(color='purple')), row=2, col=1)

# Adding trace for total balance
fig.add_trace(go.Scatter(x=n_values, y=total_balance, mode='lines', name='Total Balance', line=dict(color='brown')), row=2, col=1)

# Adding traces for amount into treasury, jackpot gains, team total, and yield
fig.add_trace(go.Scatter(x=n_values, y=amount_into_treasury_after_yield, mode='lines', name='Amount into Treasury', line=dict(color='purple')), row=3, col=1)
fig.add_trace(go.Scatter(x=n_values, y=jackpot_gains, mode='lines', name='Jackpot Gains', line=dict(color='orange')), row=3, col=1)
fig.add_trace(go.Scatter(x=n_values, y=team_total_values, mode='lines', name='Team Total', line=dict(color='brown')), row=3, col=1)
fig.add_trace(go.Scatter(x=n_values, y=yield_values, mode='lines', name='Yield', line=dict(color='green')), row=3, col=1)

# Adding traces for circulating supply, locked tokens, mystery box tokens, public tokens, and locked in NFT tokens
fig.add_trace(go.Scatter(x=n_values, y=circulating_supply, mode='lines', name='Circulating Supply', stackgroup='one', line=dict(color='cyan')), row=4, col=1)
fig.add_trace(go.Scatter(x=n_values, y=locked_tokens - mystery_box_tokens - public_tokens - locked_in_nft_values, mode='lines', name='Locked Tokens (excl. Mystery Box, Public, and NFT)', stackgroup='one', line=dict(color='magenta')), row=4, col=1)
fig.add_trace(go.Scatter(x=n_values, y=np.full_like(n_values, mystery_box_tokens), mode='lines', name='Mystery Box Tokens', stackgroup='one', line=dict(color='yellow')), row=4, col=1)
fig.add_trace(go.Scatter(x=n_values, y=np.full_like(n_values, public_tokens), mode='lines', name='Public Tokens', stackgroup='one', line=dict(color='green')), row=4, col=1)
fig.add_trace(go.Scatter(x=n_values, y=locked_in_nft_values, mode='lines', name='Locked in NFT Tokens', stackgroup='one', line=dict(color='blue')), row=4, col=1)

# Adding trace for evil address spend in each round
fig.add_trace(go.Bar(x=n_values, y=evilAddressSpendInEachRound_values, name='Evil Address Spend in Each Round', marker_color='blue'), row=5, col=1)

# Adding trace for price of $DARK in terms of $DAI
fig.add_trace(go.Scatter(x=np.arange(1, total_days + 1), y=prices, mode='lines', name='Price of $DARK in $DAI', line=dict(color='gold')), row=6, col=1)

# Adding traces for sales vs buys on each day
fig.add_trace(go.Scatter(x=np.arange(1, total_days + 1), y=buy_volumes, mode='lines', name='Buy Volume', line=dict(color='green')), row=7, col=1)
fig.add_trace(go.Scatter(x=np.arange(1, total_days + 1), y=sell_volumes, mode='lines', name='Sell Volume', line=dict(color='red')), row=7, col=1)

# Adding traces for total trade volume and cumulative volume on each day
fig.add_trace(go.Scatter(x=np.arange(1, total_days + 1), y=total_volumes, mode='lines', name='Total Trade Volume', line=dict(color='blue')), row=8, col=1)
fig.add_trace(go.Scatter(x=np.arange(1, total_days + 1), y=cumulative_volumes, mode='lines', name='Cumulative Volume', line=dict(color='orange')), row=8, col=1)

# Adding trace for buy-sell ratio
fig.add_trace(go.Scatter(x=np.arange(1, total_days + 1), y=buy_sell_ratios, mode='lines', name='Buy-Sell Ratio', line=dict(color='purple')), row=9, col=1)

# Adding traces for reserves of DARK and DAI over time
fig.add_trace(go.Scatter(x=np.arange(1, total_days + 1), y=daily_reserves_DARK, mode='lines', name='Reserves of DARK', line=dict(color='blue')), row=10, col=1)
fig.add_trace(go.Scatter(x=np.arange(1, total_days + 1), y=daily_reserves_DAI, mode='lines', name='Reserves of DAI', line=dict(color='red')), row=10, col=1)

# Adding traces for cumulative buys and sells
fig.add_trace(go.Scatter(x=np.arange(1, total_days + 1), y=cumulative_buys, mode='lines', name='Cumulative Buys', line=dict(color='green')), row=11, col=1)
fig.add_trace(go.Scatter(x=np.arange(1, total_days + 1), y=cumulative_sells, mode='lines', name='Cumulative Sells', line=dict(color='red')), row=11, col=1)

# Updating layout for the combined figure
fig.update_layout(
    height=3300,
    title_text='Combined Interactive Plots',
    showlegend=True,
    legend=dict(x=1.05, y=1),  # Position the legend outside the plot area
    hovermode='x'
)

# Show the combined figure
fig.show()

# Save the combined figure as an HTML file without opening it automatically
pio.write_html(fig, file=exports_dir, auto_open=False)

# Printing the final values
final_c = c_values[-1]
final_d = d_values[-1]
final_y = y_values[-1]
final_nfts_bought = amountOfNftsBought[-1]
final_treasury = amount_into_treasury_after_yield[-1]
final_jackpot = jackpot_gains[-1]
final_team = team_total_values[-1]
final_yield = yield_values[-1]
final_circulating = circulating_supply[-1]
final_locked = locked_tokens[-1]
final_locked_in_nft = locked_in_nft_values[-1]
final_mystery_box = mystery_box_tokens
final_public = public_tokens
final_spend_each_round = evilAddressSpendInEachRound_values[-1]
final_total_balance = total_balance[-1]

print(f"The final value of c is: {final_c:.2f}")
print(f"The final value of d is: {final_d:.2f}")
print(f"The final value of y is: {final_y:.2f}")
print(f"The final amount of NFTs bought is: {final_nfts_bought:.2f}")
print(f"The final amount into the treasury is: {final_treasury:.2f}")
print(f"The final jackpot total is: {final_jackpot:.2f}")
print(f"The final team total is: {final_team:.2f}")
print(f"The final yield is: {final_y:.2f}")
print(f"The final circulating supply is: {final_circulating:.2f}")
print(f"The final locked tokens are: {final_locked:.2f}")
print(f"The final locked in NFT tokens are: {final_locked_in_nft:.2f}")
print(f"The final mystery box tokens are: {final_mystery_box:.2f}")
print(f"The final public tokens are: {final_public:.2f}")
print(f"The final spend in each round is: {final_spend_each_round:.2f}")
print(f"The final total balance of the evil address is: {final_total_balance:.2f}")
